In [1]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_axial_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,15362724.22,-44.49,35.42,-7.53,-3.47,0.83,0.54
4,-44.49,1070042.59,-11.92,-2.61,0.63,0.18,-0.01
5,35.42,-11.92,486106.75,-5.29,1.02,1.21,0.04
6,-7.53,-2.61,-5.29,133279.44,0.37,-0.37,0.03
7,-3.47,0.63,1.02,0.37,21389.24,-0.01,0.02
8,0.83,0.18,1.21,-0.37,-0.01,1439.42,0.00
9,0.54,-0.01,0.04,0.03,0.02,0.00,62.99


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00001,0.00001,-0.00001,-0.00001,0.00001,0.00002
4,-0.00001,1.00000,-0.00002,-0.00001,0.00000,0.00000,-0.00000
5,0.00001,-0.00002,1.00000,-0.00002,0.00001,0.00005,0.00001
6,-0.00001,-0.00001,-0.00002,1.00000,0.00001,-0.00003,0.00001
7,-0.00001,0.00000,0.00001,0.00001,1.00000,-0.00000,0.00001
8,0.00001,0.00000,0.00005,-0.00003,-0.00000,1.00000,0.00001
9,0.00002,-0.00000,0.00001,0.00001,0.00001,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

83.47388839918821

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[3.37217743e+13 1.60731905e+11 3.13609732e+10 2.21377053e+09
 5.07303362e+07 1.72788798e+05 3.34038276e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999917,-0.011663,-0.005271,-0.001445,-0.000232,-0.000016,-6.482664e-07
1,0.010962,0.993365,-0.111960,-0.023607,-0.003720,-0.000250,-1.097662e-05
2,0.006390,0.109746,0.990907,-0.076759,-0.011178,-0.000748,-3.279953e-05
3,0.002178,0.031640,0.072946,0.995262,-0.055799,-0.003629,-1.585083e-04
4,0.000465,0.006677,0.014712,0.054525,0.997801,-0.034011,-1.477516e-03
5,0.000047,0.000671,0.001475,0.005391,0.033648,0.998458,-4.379349e-02
6,0.000004,0.000059,0.000129,0.000472,0.002941,0.043717,9.990395e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[8.297951170310736e-05,
 0.006635220216904658,
 0.009092648377385548,
 0.004738076223514409,
 0.002199006109893653,
 0.0015420508719069526,
 0.0009605009505830875]